# Create models and search against human proteins in Swissprot

## PSI-BLAST

Create PSSM

In [6]:
%%bash
rm ../models/profile.pssm
psiblast -subject ../data/BLAST_uniref90.fasta \
         -in_msa ../data/PF00017_seed.fasta \
         -out_pssm ../models/profile.pssm > /dev/null

Search usign this model

In [7]:
%%bash
rm ../results/psiblast_search.txt
psiblast -in_pssm ../models/profile.pssm -db ../data/SwissProt_humans_reference_all.fasta \
         -outfmt 6 -num_iterations 3 -evalue 0.001 > ../results/psiblast_search.txt

Read sequences and SH2 domain positions found by psi-blast

In [10]:
psiblast_sh2_positions = {}
with open("../results/psiblast_search.txt", 'r') as f:
    for line in f:
        if len(line)>1:
            qseqid, sseqid, pident, length, mismatch, gapopen, \
            qstart, qend, sstart, send, evalue, bitscore = line.strip().split()
            if sseqid not in psiblast_sh2_positions:
                psiblast_sh2_positions[sseqid] = [{'start':int(sstart), 'end':int(send)}]
            else:
                pos = {'start':int(sstart), 'end':int(send)}
                # check if the position has alredy been inserted
                # otherwise insert it in the dictionary
                if pos in psiblast_sh2_positions[sseqid]:
                    # position already inserted
                    continue
                else:
                    psiblast_sh2_positions[sseqid].append(pos)
        else:
            break
            
print("{} sequences found with psi-blast".format(len(psiblast_sh2_positions.keys())))

98 sequences found with psi-blast


In [12]:
psiblast_sh2_positions['P16885']

[{'start': 532, 'end': 617}, {'start': 646, 'end': 720}]

## HMM

Build HMM model

In [13]:
%%bash
hmmbuild ../models/hmm_model.hmm  ../data/PF00017_seed.fasta

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.2.1 (June 2018); http://hmmer.org/
# Copyright (C) 2018 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             ../data/PF00017_seed.fasta
# output HMM file:                  ../models/hmm_model.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     PF00017_seed            52   106    77     3.69  0.734 

# CPU time: 0.05u 0.00s 00:00:00.05 Elapsed: 00:00:00.06


Search using this model

In [14]:
%%bash
hmmsearch --domtblout ../results/hmmsearch.hmmer_domtblout \
          ../models/hmm_model.hmm \
          ../data/SwissProt_humans_reference_all.fasta > ../results/hmmsearch_results.hmmer_align

Read sequences and SH2 domain positions found by HMMER

In [15]:
hmm_sh2_positions = {}
with open("../results/hmmsearch.hmmer_domtblout") as f:
    for line in f:
        if line[0] != "#":
            target, tacc, tlen, query, qacc, qlen, \
            fevalue, fscore, fbias, _, _, dcevalue, dievalue, dscore, dbias, _, _ , \
            alifrom, alito, evfrom, envto, accuracy, desc = line.strip().split()[:23]
            tacc = target.split("|")[1]

            if tacc not in hmm_sh2_positions:
                hmm_sh2_positions[tacc] = [{'start':int(alifrom), 'end':int(alito)}]
            else:
                pos = {'start':int(alifrom), 'end':int(alito)}
                # check if the position has alredy been inserted
                # otherwise insert it in the dictionary
                if pos in hmm_sh2_positions[tacc]:
                    continue
                else:
                    hmm_sh2_positions[tacc].append(pos)

In [16]:
print("{} sequences found with psi-blast".format(len(hmm_sh2_positions.keys())))

109 sequences found with psi-blast


In [18]:
hmm_sh2_positions['P16885']

[{'start': 532, 'end': 617}, {'start': 646, 'end': 720}]

# Evaluate Models

In [32]:
from Bio import SeqIO
import math
import json

### Read reference sequences containing SH2 domain (pf00017)

Downloaded from UniProt with the query 

`database:(type:pfam pf00017) AND reviewed:yes AND organism:"Homo sapiens (Human) [9606]`

In [2]:
human_sh2 = SeqIO.parse('../data/SwissProt_humans_reference.fasta','fasta')

list_human_sh2 = []
for sequence in human_sh2:
    name = sequence.id # name is in the form sp|P46108|CRK_HUMAN
    list_human_sh2.append(name.split('|')[1])

set_human_sh2 = set(list_human_sh2)
print("There are {} human proteins containing SH2 domain in SwissProt".format(len(list_human_sh2)))

There are 100 human proteins containing SH2 domain in SwissProt


### Check all human sequences in SwissProt

Downloaded from UniProt with the query

`reviewed:yes AND organism:"Homo sapiens (Human) [9606]`

In [4]:
human = SeqIO.parse('../data/SwissProt_humans_reference_all.fasta', 'fasta')

list_human = []
for sequence in human:
    name = sequence.id # name is in the form sp|P46108|CRK_HUMAN
    list_human.append(name.split('|')[1])

print("There are {} human proteins in SwissProt".format(len(list_human)))

There are 20367 human proteins containing SH2 domain in SwissProt


Sanity check: all proteins containing SH2 domain should be in this dataset 

In [5]:
assert len(set(list_human).intersection(set_human_sh2)) == len(list_human_sh2)

### Read reference for position

This file can be downloaded from InterPro:

http://www.ebi.ac.uk/interpro/entry/pfam/PF00017/protein/reviewed/?search=Human%20#table

In the dictionary we will save also sequence length. This will be used to compute metrics later.

In [49]:
with open("../data/interpo-PF00017.json", "r") as file:
    interpro_data = json.load(file)
    
reference_sh2_positions = {}
for el in interpro_data:
    name = el['metadata']['accession']
    length = el['metadata']['length']
    if name in reference_sh2_positions:
        print("{} already in reference dict!".format(name))
        continue
    else:
        reference_sh2_positions[name] = {'length':length, 'positions':[]}
    entries = el['entries']
    for entry in entries:
        if entry['accession']=='PF00017':
            domain_locations = entry['entry_protein_locations']
            for location in domain_locations:
                if len(location['fragments'])>1:
                    print("len fragments > 1")
                position_dict = {
                    'start': location['fragments'][0]['start'],
                    'end': location['fragments'][0]['end']
                }
                reference_sh2_positions[name]['positions'].append(position_dict)

In [50]:
reference_sh2_positions['P16885']

{'length': 1265,
 'positions': [{'start': 532, 'end': 617}, {'start': 646, 'end': 720}]}

## Evaluate ability of retrieving proteins with SH2 domain

In [19]:
def computeMetrics(true_positive, true_negative, false_positive, false_negative):
    
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_negative + false_positive)
    precision = true_positive / (true_positive + false_positive)
    sensitivity = true_positive / (true_positive + false_negative)
    specificity = true_negative / (true_negative + false_positive)
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
    mcc = (true_positive*true_negative - false_positive*false_negative) / (math.sqrt(
        (true_positive + false_positive) * (true_positive + false_negative) * 
        (true_negative + false_positive) * (true_negative + false_negative)
    ))
    
    print("Accuracy: {:.2f}".format(accuracy))
    print("Precision: {:.2f}".format(precision))
    print("Sensitivity: {:.2f}".format(sensitivity))
    print("Specificity: {:.2f}".format(specificity))
    print("F1 score: {:.2f}".format(f1))
    print("MCC: {:.2f}".format(mcc)) 

In [21]:
def evaluateSequencesSH2(retrieved_sequences, ground_truth):
    """
    retrieved_sequences = dict of the type {'P16885':[{'start': 532, 'end': 617}], ...}
    ground_truth = set of human sequences containing SH2 (set_human_sh2 in this notebook)
    """
    retrieved_sequences = set(retrieved_sequences.keys())
    
    true_positive = len(set(retrieved_sequences).intersection(ground_truth))
    # If there aren't true postive later there will be a division by 0
    if true_positive == 0:
        print("No true positive!")
        return
    
    false_negative = len(ground_truth) - true_positive
    false_positive = len(set(retrieved_sequences)) - true_positive
    true_negative = len(list_human) - true_positive - false_positive - false_negative
    
    computeMetrics(
        true_positive,
        true_negative,
        false_positive,
        false_negative
    )

In [27]:
# Evaluate PSI-BLAST
evaluateSequencesSH2(psiblast_sh2_positions, set_human_sh2)

Accuracy: 1.00
Precision: 1.00
Sensitivity: 0.98
Specificity: 1.00
F1 score: 0.99
MCC: 0.99


In [31]:
# Evaluate HMM
evaluateSequencesSH2(hmm_sh2_positions, set_human_sh2)

Accuracy: 1.00
Precision: 0.92
Sensitivity: 1.00
Specificity: 1.00
F1 score: 0.96
MCC: 0.96


## Evaluate ability of matching domains

In [39]:
#                                        SH2                                SH2
#reference (truth):              |||||||||||||||||||||             |||||||||||||||||||||||||
#model (predicted):    /////    |||||||||||||||||||||||        ||||||||||||||||

In [40]:
def createPositionSet(positions_list):
    """
    position_list = [{'start': '532', 'end': '617'}, {'start': '646', 'end': '720'}]
    return a list {532, 533, ...., 617, 646, ..., 720}
    """
    positions = set()
    for pos in positions_list:
        p = [i for i in range(pos['start'], pos['end']+1)]
        positions = positions.union(p)
    return positions

In [56]:
def evaluatePositionsSH2(predicted_positions_sh2, reference_positions_sh2):
    """
    predicted_positions_sh2: dict of the type {'P16885':[{'start': 532, 'end': 617}], ...}
    reference_positions_sh2 {'P16885':{'length': 1265, 'positions': [{'start': 532, 'end': 617}]}, ...}
    """
    list_tp = []
    list_fp = []
    list_fn = []
    list_tn = []

    for seqid in predicted_positions_sh2:

        if seqid in reference_positions_sh2:
            sequence_length = reference_positions_sh2[seqid]['length']
            reference_positions = createPositionSet(reference_positions_sh2[seqid]['positions'])
            identified_positions = createPositionSet(predicted_positions_sh2[seqid])    

            overlap  = reference_positions.intersection(identified_positions)

            true_positive = len(overlap)
            false_positive = len(identified_positions.difference(overlap))
            false_negative = len(reference_positions.difference(overlap))
            true_negative  = sequence_length - true_positive - false_positive - false_negative

            list_tp.append(true_positive)
            list_fp.append(false_positive)
            list_fn.append(false_negative)
            list_tn.append(true_negative)

        #else:
        #    print("Error! sequence not found in the reference dataset -- False positive")   
        #    break
            
    computeMetrics(
        true_positive=sum(list_tp),
        true_negative=sum(list_tn),
        false_positive=sum(list_fp),
        false_negative=sum(list_fn)
    )

In [54]:
# evaluate psiblast
evaluatePositionsSH2(psiblast_sh2_positions, reference_sh2_positions)

Accuracy: 1.00
Precision: 0.98
Sensitivity: 1.00
Specificity: 1.00
F1 score: 0.99
MCC: 0.99


In [58]:
# evaluate hmm
evaluatePositionsSH2(hmm_sh2_positions, reference_sh2_positions)

Accuracy: 1.00
Precision: 0.99
Sensitivity: 1.00
Specificity: 1.00
F1 score: 1.00
MCC: 0.99
